<a href="https://colab.research.google.com/github/NolanDeGeare/Natural-Language-to-SQL/blob/main/text2sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()  #this uploads my csv file to collab


Saving all_seasons2.csv to all_seasons2.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import os

excel_file_path = "/content/drive/My Drive/IT 244/text2sql_log.xlsx"




In [ ]:
import sqlite3


#reads the CSV file into a pandas dataframe
df = pd.read_csv("all_seasons2.csv")

#this checks the data
df.head()


,player_id,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Randy Livingston,HOU,22,193.04,94.800728,Louisiana State,USA,1996,2,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,1,Gaylon Nickerson,WAS,28,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,2,George Lynch,VAN,26,203.20,103.418976,North Carolina,USA,1993,1,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,3,George McCloud,LAL,30,203.20,102.058200,Florida State,USA,1989,1,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,4,George Zidek,DEN,23,213.36,119.748288,UCLA,USA,1995,1,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97


In [ ]:
#creates a SQLite database file
conn = sqlite3.connect("nba_data.db")
cursor = conn.cursor()

#converts to an SQL table
df.to_sql("all_seasons2", conn, if_exists="replace", index=False)

print("Data loaded")


Data loaded


In [ ]:
#testing to see if data is uploaded and able to be retrived
query = "SELECT player_name FROM all_seasons2 ORDER BY age DESC"
df_query = pd.read_sql_query(query, conn)

print(df_query)



           player_name
0         Kevin Willis
1        Robert Parish
2      Dikembe Mutombo
3         Vince Carter
4        Udonis Haslem
...                ...
12839       AJ Griffin
12840      Jalen Duren
12841  Jermaine O'Neal
12842      Kobe Bryant
12843     Andrew Bynum

[12844 rows x 1 columns]


In [ ]:
%%capture
!pip install transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch


In [ ]:
#loads text2sql model and the tokenizer from huggingface
model_name = "gaussalgo/T5-LM-Large-text2sql-spider"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
def generate_sql(natural_language_query):
    schema = """
    Table: all_seasons2
    Columns:
    - player_id (INTEGER)
    - player_name (TEXT)
    - team_abbreviation (TEXT)
    - age (INTEGER)
    - player_height (FLOAT)
    - player_weight (FLOAT)
    - college (TEXT)
    - country (TEXT)
    - draft_year (INTEGER)
    - draft_round (INTEGER)
    - pts (FLOAT)
    - reb (FLOAT)
    - ast (FLOAT)
    - net_rating (FLOAT)
    - oreb_pct (FLOAT)
    - dreb_pct (FLOAT)
    - usg_pct (FLOAT)
    - ts_pct (FLOAT)
    - ast_pct (FLOAT)
    - season (TEXT, e.g., '1996-97')

    Write a SQL query that accurately retrieves data from the all_seasons table using WHERE conditions as needed.
    - Use COUNT(*) when aggregating data.
    - Use ORDER BY COUNT(*) DESC when ranking grouped data.
    - Ensure all column names are spelled correctly.
    - When using LIMIT add an amount to the end e.g LIMIT 10.
    - Ensure using things like COUNT, are always capital.
    - Replace "along with" with ",".
    - Seperate data with ",".
    - DO NOT use "-" in the query.
    """


    #formatting input for the model
    input_text = f"### Schema: {schema} \n### Question: {natural_language_query} \n### SQL Query:"

    #converts input text into tokens for the model to understand
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    #this generates the sql query
    with torch.no_grad():
        outputs = model.generate(**inputs)

    #turning the generated output into a sql query that is able to be exectuted
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query




In [ ]:
def save_query_to_excel(nl_query, sql_query, file_path=excel_file_path):
    """Appends a new NL query and SQL query to an existing or new Excel file."""

    #checks to see if the csv file already exists
    if os.path.exists(file_path):
        #if the file exist it just adds to it
        df = pd.read_excel(file_path)
    else:
        #if it doesnt exist it creates it
        df = pd.DataFrame(columns=["Natural Language Query", "Generated SQL Query"])

    #adding a new row for next input and output to be logged
    new_data = pd.DataFrame([[nl_query, sql_query]], columns=df.columns)
    df = pd.concat([df, new_data], ignore_index=True)

    #saves to the excel file
    df.to_excel(file_path, index=False)

    print("Query has been logged")



In [ ]:
# submiting a query
nl_query = "give me the player_name, team_abbreviation for all players"
sql_query = generate_sql(nl_query)
print("Generated SQL Query:", sql_query)

# logs the query
save_query_to_excel(nl_query, sql_query)

Generated SQL Query: SELECT player_name - team_abbreviation FROM all_seasons
Query has been logged


In [ ]:
#function to execute the query
def execute_sql_query(sql_query, conn):

    try:
        df_result = pd.read_sql_query(sql_query, conn)
        return df_result
    except Exception as e:
        print("Error executing query:", e)
        return None


In [ ]:
#checks to see if query worked, if it did it displays the results
df_results = execute_sql_query(sql_query, conn)

if df_results is not None:
    print("Query Results:")
    print(df_results)
else:
    print("No results returned.")

Query Results:
       player_id       player_name team_abbreviation  age  player_height  \
0              0  Randy Livingston               HOU   22         193.04   
1              1  Gaylon Nickerson               WAS   28         190.50   
2              2      George Lynch               VAN   26         203.20   
3              3    George McCloud               LAL   30         203.20   
4              4      George Zidek               DEN   23         213.36   
...          ...               ...               ...  ...            ...   
12839      12839       Joel Embiid               PHI   29         213.36   
12840      12840   John Butler Jr.               POR   20         213.36   
12841      12841      John Collins               ATL   25         205.74   
12842      12842      Jericho Sims               NYK   24         208.28   
12843      12843    JaMychal Green               GSW   33         205.74   

       player_weight                college   country draft_year draft_r